In [10]:
import pandas as pd
import ast

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
import joblib

df = pd.read_csv("../data/games_extended.csv")

print("Shape carregado do CSV:", df.shape)
df.head()

Shape carregado do CSV: (400, 3)


,name,description,genres
0,Grand Theft Auto V,"Rockstar Games went bigger, since their previo...",['Strategy']
1,The Witcher 3: Wild Hunt,"The third game in a series, it holds nothing b...","['Action', 'RPG']"
2,Portal 2,Portal 2 is a first-person puzzle game develop...,"['Shooter', 'Puzzle']"
3,Counter-Strike: Global Offensive,Counter-Strike is a multiplayer phenomenon in ...,['Shooter']
4,Tomb Raider (2013),A cinematic revival of the series in its actio...,['Action']


In [11]:
def parse_genres(genres_obj):
    """
    Garante que 'genres_obj' seja uma lista de strings.
    Pode vir como lista mesmo ou como string tipo "['Action', 'RPG']".
    """
    if isinstance(genres_obj, list):
        return genres_obj
    if isinstance(genres_obj, str):
        try:
            parsed = ast.literal_eval(genres_obj)
            if isinstance(parsed, list):
                return parsed
        except Exception:
            pass
    return []

def clean_genre(genres_obj):
    """
    Remove 'Action' quando aparece junto com outros gêneros
    e retorna o primeiro gênero restante.
    Se não sobrar nada, retorna 'Unknown'.
    """
    genres_list = parse_genres(genres_obj)

    if "Action" in genres_list and len(genres_list) > 1:
        genres_list = [g for g in genres_list if g != "Action"]

    if len(genres_list) > 0:
        return genres_list[0]

    return "Unknown"

In [12]:
df["description"] = df["description"].fillna("")
df["genre"] = df["genres"].apply(clean_genre)

# remove descrições muito curtas
df = df[df["description"].str.len() > 10]

print("Shape depois de limpar descrição e gênero:", df.shape)
print("\nDistribuição de gêneros (limpos):")
print(df["genre"].value_counts())

df[["name", "genre"]].head()

Shape depois de limpar descrição e gênero: (400, 4)

Distribuição de gêneros (limpos):
genre
Shooter                  123
Adventure                102
RPG                       64
Action                    39
Strategy                  23
Indie                     19
Simulation                11
Arcade                     4
Racing                     3
Sports                     3
Platformer                 3
Casual                     2
Massively Multiplayer      2
Puzzle                     1
Fighting                   1
Name: count, dtype: int64


,name,genre
0,Grand Theft Auto V,Strategy
1,The Witcher 3: Wild Hunt,RPG
2,Portal 2,Shooter
3,Counter-Strike: Global Offensive,Shooter
4,Tomb Raider (2013),Action


In [13]:
counts = df["genre"].value_counts()
print("Distribuição completa:")
print(counts)

MAX_GENRES = 5  # queremos até 5 gêneros diferentes
top_genres = counts.head(MAX_GENRES).index

print("\nTop gêneros selecionados:", list(top_genres))

df = df[df["genre"].isin(top_genres)]

print("\nDistribuição após manter apenas top gêneros:")
print(df["genre"].value_counts())
print("\nShape após filtro de gêneros:", df.shape)
print("Gêneros únicos finais:", df["genre"].unique())

Distribuição completa:
genre
Shooter                  123
Adventure                102
RPG                       64
Action                    39
Strategy                  23
Indie                     19
Simulation                11
Arcade                     4
Racing                     3
Sports                     3
Platformer                 3
Casual                     2
Massively Multiplayer      2
Puzzle                     1
Fighting                   1
Name: count, dtype: int64

Top gêneros selecionados: ['Shooter', 'Adventure', 'RPG', 'Action', 'Strategy']

Distribuição após manter apenas top gêneros:
genre
Shooter      123
Adventure    102
RPG           64
Action        39
Strategy      23
Name: count, dtype: int64

Shape após filtro de gêneros: (351, 4)
Gêneros únicos finais: ['Strategy' 'RPG' 'Shooter' 'Action' 'Adventure']


In [14]:
class_counts = df["genre"].value_counts()
print("Contagem original por gênero:")
print(class_counts)

MAX_SAMPLES_PER_CLASS = 80

min_count = min(class_counts.min(), MAX_SAMPLES_PER_CLASS)
print("\nTentando usar", min_count, "exemplos por classe.")

# Se alguma classe tiver muito pouco, ainda assim seguimos (vai ficar pequeno mesmo)
balanced_df = (
    df.groupby("genre", group_keys=False)
      .apply(lambda g: g.sample(min_count, random_state=42))
      .reset_index(drop=True)
)

print("\nShape do dataset balanceado:", balanced_df.shape)
print(balanced_df["genre"].value_counts())

Contagem original por gênero:
genre
Shooter      123
Adventure    102
RPG           64
Action        39
Strategy      23
Name: count, dtype: int64

Tentando usar 23 exemplos por classe.

Shape do dataset balanceado: (115, 4)
genre
Action       23
Adventure    23
RPG          23
Shooter      23
Strategy     23
Name: count, dtype: int64


C:\Users\cttcl\AppData\Local\Temp\ipykernel_2336\2018461160.py:13: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: g.sample(min_count, random_state=42))


In [15]:
X = balanced_df["description"]
y = balanced_df["genre"]

n_samples = len(y)
n_classes = y.nunique()

print("Amostras totais:", n_samples)
print("Nº de classes:", n_classes)
print("Gêneros presentes:", y.unique())

# test_size automático:
# precisa garantir que nº de amostras no teste >= nº de classes
base_test_size = 0.2
min_test_size = (n_classes + 1) / n_samples  # +1 de folga

test_size = max(base_test_size, min_test_size)
print(f"\nUsando test_size = {test_size:.3f}")

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=test_size,
    random_state=42,
    stratify=y
)

print("\nTamanho treino:", len(X_train))
print("Tamanho teste:", len(X_test))
print("\nDistribuição no treino:")
print(pd.Series(y_train).value_counts())
print("\nDistribuição no teste:")
print(pd.Series(y_test).value_counts())

Amostras totais: 115
Nº de classes: 5
Gêneros presentes: ['Action' 'Adventure' 'RPG' 'Shooter' 'Strategy']

Usando test_size = 0.200

Tamanho treino: 92
Tamanho teste: 23

Distribuição no treino:
genre
Strategy     19
Adventure    19
RPG          18
Action       18
Shooter      18
Name: count, dtype: int64

Distribuição no teste:
genre
Action       5
RPG          5
Shooter      5
Adventure    4
Strategy     4
Name: count, dtype: int64


In [16]:
vectorizer = TfidfVectorizer(stop_words="english", max_features=10000)
X_train_vec = vectorizer.fit_transform(X_train)

clf = LinearSVC(class_weight="balanced", random_state=42)
clf.fit(X_train_vec, y_train)

,penalty,'l2'
,loss,'squared_hinge'
,dual,'auto'
,tol,0.0001
,C,1.0
,multi_class,'ovr'
,fit_intercept,True
,intercept_scaling,1
,class_weight,'balanced'
,verbose,0
,random_state,42


In [17]:
X_test_vec = vectorizer.transform(X_test)
accuracy = clf.score(X_test_vec, y_test)
print("Acurácia no conjunto de teste:", accuracy)

y_pred = clf.predict(X_test_vec)
print("\nClassification report:")
print(classification_report(y_test, y_pred))

Acurácia no conjunto de teste: 0.391304347826087

Classification report:
              precision    recall  f1-score   support

      Action       0.50      0.40      0.44         5
   Adventure       0.33      0.25      0.29         4
         RPG       0.30      0.60      0.40         5
     Shooter       0.00      0.00      0.00         5
    Strategy       0.60      0.75      0.67         4

    accuracy                           0.39        23
   macro avg       0.35      0.40      0.36        23
weighted avg       0.34      0.39      0.35        23



In [18]:
import pathlib

model_dir = pathlib.Path("../model")
model_dir.mkdir(exist_ok=True)

joblib.dump(clf, model_dir / "classifier.pkl")
joblib.dump(vectorizer, model_dir / "vectorizer.pkl")

print("Modelo salvo em:", model_dir / "classifier.pkl")
print("Vetorizador salvo em:", model_dir / "vectorizer.pkl")

Modelo salvo em: ..\model\classifier.pkl
Vetorizador salvo em: ..\model\vectorizer.pkl
